In [ ]:
# http://lottoscar.altervista.org/archivi.html
# https://www.simplifiedpython.net/python-download-file/#Download_Zip_File 
# https://living-sun.com/it/python/705402-how-can-i-extract-a-closed-zip-with-python-python-zipfile.html


In [1]:
# importazione delle librerie
import requests
import zipfile
import csv
import pandas as pd
import numpy as np
import funzioni as fn
from pprint import pprint
from itertools import combinations


CLASSI

In [43]:
class Numero:

    def __init__(self, n, dataset):

        self.precedente_ritardo = 0
        self.frequenza = 0
        self.numero = n
        self.dataset = dataset
        self.ruota = dataset[1][1]
        self.ritardi = []
        self.scompensi = []
        self.scomp = 0

    def info(self):
        temp = []
        #print(self.dataset[0][2:])
        for i in range(len(self.dataset)):
            #print(self.dataset[i][2:])
            if str(self.numero) in self.dataset[i][2:]:
                temp.append(i+1)

        if len(temp) == 0:
            self.precedente_ritardo = len(self.dataset) + 1
            self.ritardo_max = self.precedente_ritardo
            self.frequenza = 0
            self.previsione = np.NaN
            self.scomp = np.NaN
        else:
            self.precedente_ritardo = temp[len(temp)-1] - temp[len(temp)-2] # Rutaro precedente all'ultima uscita
            self.ultimo_ritardo = len(self.dataset) - temp[len(temp)-1]     # ritardo all'ultima uscita
        
            for i in range(1, len(temp)):
                r = temp[i] - temp[i-1]                 # Ritardo del numero nel momento in cui è uscito
                self.ritardi.append(r)                  # archivio dei ritardi
                n_estr = temp[i]                        # numero della estrazione del numero nel momento in cui è uscito
                freq = int(n_estr / len(temp[:i]))      # frequenza nel momento in cui è uscito
                self.scomp += (r - freq)                # scompensazione nel momento in cui è uscito
                self.scompensi.append(self.scomp)

            self.ritardo_max = max(self.ritardi)                        # massimo ritardo fatto dal numero
            self.frequenza = int((len(self.dataset)+1)/(len(temp)+1))   # frequenza attuale
            self.previsione = self.frequenza - self.ultimo_ritardo      # previsione di uscita calcolata sulla frequenza e il ritardo attiale

    def mostra_info(self):

        #valori = {
        #    "Numero" : self.numero,
        #    "Ruota" : self.ruota,
        #    "Ritardo Precedente" : self.precedente_ritardo,
        #    "Ritardo Attuale" : self.ultimo_ritardo,
        #    "Ritardo Massimo" : self.ritardo_max,
        #    "Frequenza" : self.frequenza,
        #    "Previsione" : self.previsione,
        #    "Scompensazione": self.scomp
        #}

        valori = [
            self.numero,
            self.ruota,
            self.precedente_ritardo,
            self.ultimo_ritardo,
            self.ritardo_max,
            self.frequenza,
            self.previsione,
            self.scomp
            ]

        return valori


In [3]:
class Ambi:

    def __init__(self, n, dataset):

        self.ritardo = 0
        self.frequenza = 0
        self.ambo = (str(n[0]), str(n[1]))              # come un turple (a, b)
        self.dataset = dataset
        self.ruota = dataset[1][1]
        self.ritardi = []

    def info(self):
        temp = []
        for i in range(len(self.dataset)):
            comb = list(combinations(self.dataset[i][2:], 2))
            #print(comb)
            if self.ambo in comb:
                temp.append(i+1)
        if len(temp) == 0:
            self.ritardo = len(self.dataset) + 1
            self.ritardo_max = self.ritardo
            self.frequenza = 0
            self.previsione = "N/A"
        else:
            self.ritardo = temp[len(temp)-1] - temp[len(temp)-2]
            self.ultimo_ritardo = len(self.dataset) - temp[len(temp)-1]
        
            for i in range(1, len(temp)):
                self.ritardi.append(temp[i] - temp[i-1])

            self.ritardo_max = max(self.ritardi)
            self.ritardo_medio = int(sum(self.ritardi)/len(self.ritardi))
            self.frequenza = int((len(self.dataset)+1)/(len(temp)+1))
            self.previsione = self.frequenza - self.ultimo_ritardo

    def mostra_info(self):

        valori = {
            "Ambo" : self.ambo,
            "Ruota" : self.ruota,
            "Ultimo Ritardo" : self.ritardo,
            "Ritardo Attuale" : self.ultimo_ritardo,
            "Ritardo Massimo" : self.ritardo_max,
            "Media Ritardi" : self.ritardo_medio,
            "Frequenza" : self.frequenza,
            "Previsione" : self.previsione
        }

        return valori
    

FUNZIONI

In [63]:
# Funzione per scaricare il database dei numeri
def update_archivio():
    # download archivio
    url = "http://lottoscar.altervista.org/ArchivioLotto.italia.zip"
    r = requests.get(url)
    with open("ArchivioLotto.italia.zip", "wb") as zip:
        zip.write(r.content)

    # estrazione del file zip
    f = zipfile.ZipFile("ArchivioLotto.italia.zip")
    f.extractall()
    # elaborazione delle singole ruote dal file generale
    fn.estrai_ruote('ArchivioLotto.italia.csv')


In [67]:
# Funzione per creare una tabella con tutte le statistiche di ogni numero
def print_numeri(data):
    numeri = []
    for n in range(1, 91):
        t = Numero(n, data)
        t.info()
        numeri.append(t.mostra_info())
    
    return numeri


In [64]:
# Funzione per creare una tabella con tutte le statistiche di ogni numero
def print_dataframe_from_numeri(data):
    numeri = print_numeri(data)
    df = pd.DataFrame(numeri, columns=[
                                        "Numero", 
                                        "Ruota", 
                                        "Ritardo Precedente",
                                        "Ritardo Attuale",
                                        "Ritardo Massimo", 
                                        "Frequenza", 
                                        "Previsione",
                                        "Scompensazione"
                                        ]
                    ).set_index(
                                [
                                    "Numero", 
                                    "Ruota"
                                ]
                            )

    return df

CODICE

In [62]:
update_archivio()
data = fn.leggi_da_csv("BA.csv")

NameError: name 'update_archivio' is not defined

In [68]:
database = print_numeri(data)
print(database)


[[1, 'BA', 20, 22, 91, 17, -5, 650], [2, 'BA', 5, 1, 105, 18, 17, 153], [3, 'BA', 13, 6, 107, 17, 11, -384], [4, 'BA', 3, 2, 99, 16, 14, 33], [5, 'BA', 98, 10, 109, 17, 7, 190], [6, 'BA', 9, 33, 111, 19, -14, 484], [7, 'BA', 22, 21, 147, 19, -2, -233], [8, 'BA', 26, 23, 107, 17, -6, -40], [9, 'BA', 5, 11, 79, 17, 6, 388], [10, 'BA', 9, 17, 131, 17, 0, 32], [11, 'BA', 5, 1, 97, 18, 17, -77], [12, 'BA', 7, 26, 95, 17, -9, -532], [13, 'BA', 13, 21, 96, 18, -3, 39], [14, 'BA', 14, 2, 152, 17, 15, -104], [15, 'BA', 26, 3, 111, 16, 13, -67], [16, 'BA', 5, 12, 121, 18, 6, 261], [17, 'BA', 3, 14, 125, 18, 4, 263], [18, 'BA', 5, 28, 95, 18, -10, 726], [19, 'BA', 10, 29, 91, 18, -11, 155], [20, 'BA', 7, 4, 134, 16, 12, 251], [21, 'BA', 22, 16, 202, 19, 3, 234], [22, 'BA', 3, 7, 115, 17, 10, 328], [23, 'BA', 2, 0, 114, 17, 17, -198], [24, 'BA', 1, 13, 117, 20, 7, 77], [25, 'BA', 16, 11, 141, 20, 9, -146], [26, 'BA', 6, 13, 106, 16, 3, 142], [27, 'BA', 41, 6, 108, 18, 12, -84], [28, 'BA', 35, 0, 1

In [ ]:
lista_numeri = []
for i in numeri:
    #print(i)
    p = int(i[0]["Previsione"])
    r = int(i[0]["Ritardo Attuale"]) < int(i[0]["Frequenza"])
    f = int(i[0]["Frequenza"])
    if p and (p < 10 and p > 0) and f < 20:
        #print(i[0]["Numero"])
        lista_numeri.append(i[0]["Numero"])
        print(i)

print(lista_numeri)


In [10]:
ambi = []
comb = list(combinations(lista_numeri, 2))
for a in comb:
    t = Ambi(a, data[1:])
    t.info()
    ambi.append([t.mostra_info()])

#print(ambi[0])
for i in ambi:
    a = int(i[0]["Ritardo Attuale"]) != int(i[0]["Ritardo Massimo"])
    b = int(i[0]["Ritardo Attuale"]) < int(i[0]["Frequenza"])
    d = int(i[0]["Previsione"])
    if a and b and d < 20:
        print(i)

[{'Ambo': ('17', '65'), 'Ruota': 'BA', 'Ultimo Ritardo': 223, 'Ritardo Attuale': 718, 'Ritardo Massimo': 3095, 'Media Ritardi': 822, 'Frequenza': 733, 'Previsione': 15}]


In [22]:
import pandas_ta as ta
import matplotlib.pyplot as plt

df = pd.DataFrame()

temp = Numero(1, data)
temp.info()
n = str("close")
df[n] = temp.ritardi
sma10 = df.ta.sma(length=10)
sma30 = df.ta.sma(length=30)
df = pd.concat([df[n], sma10, sma30], axis=1)
df['SMA_sigLine'] = np.where(df['SMA_10'] > df['SMA_30'], 1.0, 0.0 )
df.to_csv("stats.csv")
print(df)

     close  SMA_10     SMA_30  SMA_sigLine
0       12     NaN        NaN          0.0
1       15     NaN        NaN          0.0
2        9     NaN        NaN          0.0
3       22     NaN        NaN          0.0
4       27     NaN        NaN          0.0
..     ...     ...        ...          ...
368      3    17.9  18.466667          0.0
369      4    16.3  17.966667          0.0
370     14    17.3  18.000000          0.0
371     56    22.6  19.033333          1.0
372     20    19.8  19.166667          1.0

[373 rows x 4 columns]


In [81]:
def plot_graph(df):

    df=df.astype(float)
    plt.rcParams["figure.figsize"] = (36, 30)
    #plt.rcParams["savefig.format"] = 'png'  

    # plotting di tutte le curve 
    plt.plot(df['close'], color='blue', label='ritardo')

    # configuraione degli assi
    plt.xlabel('Numero uscita',fontsize=18)
    plt.ylabel('Riardo',fontsize=18)

    # plot signals EMA_RSI_ATR
    #plt.plot(df.index, df['BuyEMA_RSI_ATR'], '^', markersize = 15, color = 'purple', label='BuyEMA_RSI_ATR') # plot ‘Buy’ signals
    #plt.plot(df.index, df['SellEMA_RSI_ATR'], 'v', markersize = 15, color = 'black', label='SellEMA_RSI_ATR') # plot ‘Sell’ signals

    # configurazioni del titolo, leggende, griglia
    plt.title('GRAFICO', fontsize = 20)
    plt.legend()
    plt.grid()
    plt.show()

In [85]:
#plot_graph(df)